In [ ]:
import pandas as pd
position=pd.read_csv('position.csv')
val=[1,2,3,4,5,6]
input_df=[]
VH_part=[]
VL_part=[]
for i in range(0,len(val)):
    df=pd.read_csv(f"BI_master_neg_part{val[i]}_aligned.csv")
    input_df.append(df)
    VH_part.append(df.iloc[:,df.T.index.tolist().index('H1'):df.T.index.tolist().index('H113')+1])
    VL_part.append(df.iloc[:,df.T.index.tolist().index('L1'):df.T.index.tolist().index('L107')+1])

VH_position=position.T.index[-134:]
VL_position=position.T.index[1:-134]
po_VL=[]
po_VH=[]
VL_position_list=list(VL_position)
VH_position_list=list(VH_position)
for i in range(0,len(val)):
    po_VL.append(list(VL_part[i].T.index))
    po_VH.append(list(VH_part[i].T.index))

In [ ]:
## Extra sites for VL region
import numpy as np
extra_site_VL=[]
extra_site_VH=[]
seq_drop_VL=[]
seq_drop_VH=[]
for i in range(0,6):
    print(i)
    extra_site_VL_p1=[ x for x in np.setdiff1d(po_VL[i],VL_position_list) if "27" not in x ]
    extra_site_VL.append(extra_site_VL_p1)
    seq_drop_VL.append(VL_part[i].replace('-','')[extra_site_VL_p1].apply(lambda x: ''.join(x.dropna().astype(str)),axis=1))
    extra_site_VH_p1=[ x for x in np.setdiff1d(po_VH[i],VH_position_list) if "35" not in x ]
    extra_site_VH_p2=[ x for x in extra_site_VH_p1 if "100" not in x ]
    extra_site_VH.append(extra_site_VH_p2)
    seq_drop_VH.append(VH_part[i].replace('-','')[extra_site_VH_p2].apply(lambda x: ''.join(x.dropna().astype(str)),axis=1))

number_VL=[]
number_VH=[]
for j in range(0,6):
    mab_num=[]
    for i in range(0,len(seq_drop_VL[j])):
        if len(seq_drop_VL[j][i])>0:
            mab_num.append(i)
    number_VL.append(mab_num)

for j in range(0,6):
    mab_num=[]
    for i in range(0,len(seq_drop_VH[j])):
        if len(seq_drop_VH[j][i])>0:
            mab_num.append(i)
    number_VH.append(mab_num)

In [ ]:
Fv_part=[]
for i in range(0,6):
    VH_part_eli=VH_part[i].drop(number_VH[i], axis=0).dropna()
    VL_part_eli=VL_part[i].drop(number_VL[i], axis=0).dropna()
    VH_part_eliminate=VH_part_eli.loc[:, ~(VH_part_eli.astype(str) == '-').all()]
    VL_part_eliminate=VL_part_eli.loc[:, ~(VL_part_eli.astype(str) == '-').all()]
    Fv_part.append(pd.concat([VH_part_eliminate,VL_part_eliminate],axis=1,sort=False).dropna().reset_index().iloc[:,1:])
    

In [ ]:
## Rename VL chain 

l=['L27','L28','L29','L30']
L1_temp=['L27','L28','L29','L30','L30A','L30B','L30C','L30D','L30E','L30F','L30G','L30H','L30I','L30J','L30K','L30L']
VL_all=[]
for num in range(0,6):
    print(num)
    L1_rename=[]
    VL_F1=[]
    VL_F3=[]
    for i in range(0,4):
        L1_rename+=list(filter(lambda a: l[i] in a, list(Fv_part[num].T.index)))

    L1_temp[:len(L1_rename)]
    CDRL1=Fv_part[num][L1_rename]
    result_CDRL1=[]
    for i in range(0,len(CDRL1)):
        #print(i)
        test_list=list(CDRL1.iloc[i,:])
        test_list.sort(key = '-'.__eq__) 
        result_CDRL1.append(test_list)
    CDRL1_align=pd.DataFrame(result_CDRL1)
    CDRL1_align.columns=L1_temp[:len(CDRL1.T)] 

    vl_start=Fv_part[num].T.index.tolist().index('L1')
    vl_constat1=Fv_part[num].T.index.tolist().index('L26')
    vl_constat2=Fv_part[num].T.index.tolist().index('L31')
    vl_end=Fv_part[num].T.index.tolist().index('L107')
    vl_extra1=CDRL1_align.T.index.tolist().index('L30F')

    VL_F1=Fv_part[num].iloc[:,vl_start:vl_constat1+1]
    VL_F3=Fv_part[num].iloc[:,vl_constat2:vl_end+1]
    VL_align=[]
    VL_chain=[]

    if len(CDRL1_align.T)>10:
        CDRL1_extra=CDRL1_align.iloc[:,vl_extra1+1:]
        CDRL1_extrav2=CDRL1_extra.replace('-','').loc[:, ~(CDRL1_extra.replace('-','').astype(str) == '-').all()]
        CDRL1_extrav3=CDRL1_extrav2.apply(lambda x: ''.join(x.dropna().astype(str)),axis=1)
        mab_num_cdrl1=[]
        for i in range(0,len(CDRL1_extrav3)):
            if len(CDRL1_extrav3[i])>0:
                mab_num_cdrl1.append(i)
        VL_chain=pd.concat([VL_F1,CDRL1_align,VL_F3],axis=1,sort=False).drop(mab_num_cdrl1, axis=0).dropna()
        VL_align=VL_chain.loc[:, ~(VL_chain.astype(str) == '-').all()]
    else:
        VL_chain=pd.concat([VL_F1,CDRL1_align,VL_F3],axis=1,sort=False).dropna()
        VL_align=VL_chain.loc[:, ~(VL_chain.astype(str) == '-').all()]
    VL_all.append(VL_align)

In [ ]:
## Rename VH chain 

h1=['H31','H32','H33','H34','H35']
H1_temp=['H31','H31A','H31B','H32','H33','H34','H35','H35A','H35B','H35C','H35D','H35E','H35F','H35G','H35H','H35I','H35J','H35K']
VH_all=[]
for num in range(0,6):
    H1_rename=[]
    print(num)
    for i in range(0,5):
        H1_rename+=list(filter(lambda a: h1[i] in a, list(Fv_part[num].T.index)))

    H1_temp[:len(H1_rename)]
    CDRH1=Fv_part[num][H1_rename]
    result_CDRH1=[]
    for i in range(0,len(CDRH1)):
        #print(i)
        test_list=list(CDRH1.iloc[i,:])
        test_list.sort(key = '-'.__eq__) 
        result_CDRH1.append(test_list)
    CDRH1_align=pd.DataFrame(result_CDRH1)
    CDRH1_align.columns=H1_temp[:len(CDRH1.T)] 

    vh_start=Fv_part[num].T.index.tolist().index('H1')
    vh_constat1=Fv_part[num].T.index.tolist().index('H30')
    vh_constat2=Fv_part[num].T.index.tolist().index('H36')
    vh_constat3=Fv_part[num].T.index.tolist().index('H92')
    vh_constat4=Fv_part[num].T.index.tolist().index('H103')
    vh_end=Fv_part[num].T.index.tolist().index('H113')
    vh_extra1=CDRH1_align.T.index.tolist().index('H35')

    VH_F1=Fv_part[num].iloc[:,vh_start:vh_constat1+1]
    VH_F2=Fv_part[num].iloc[:,vh_constat2:vh_constat3+1]
    VH_F3=Fv_part[num].iloc[:,vh_constat4:vh_end+1]
    CDRH3=Fv_part[num].T.iloc[Fv_part[num].T.index.tolist().index('H92')+1:Fv_part[num].T.index.tolist().index('H103')]
    if len(CDRH1_align.T)>7:
        CDRH1_extra=CDRH1_align.iloc[:,vh_extra1+1:]
        CDRH1_extrav2=CDRH1_extra.replace('-','').loc[:, ~(CDRH1_extra.replace('-','').astype(str) == '-').all()]
        CDRH1_extrav3=CDRH1_extrav2.apply(lambda x: ''.join(x.dropna().astype(str)),axis=1)
        ## cdrh1 drop row: mab_num_cdrh1
        mab_num_cdrh1=[]
        for i in range(0,len(CDRH1_extrav3)):
            if len(CDRH1_extrav3[i])>0:
                mab_num_cdrh1.append(i)

        CDRH3=Fv_part[num].T.iloc[Fv_part[num].T.index.tolist().index('H92')+1:Fv_part[num].T.index.tolist().index('H103')]
        result_CDRH3=[]
        CDRH3_extra=[]
        CDRH3_noextra=[]
        for i in range(len(CDRH3.columns)):
            CDRH3_list=list(CDRH3.iloc[:,i])
            CDRH3_list=[s for s in CDRH3_list if s != '-']
            if len(CDRH3_list)<=23:
                CDRH3_noextra.append(i)
                for elem in range(23-len(CDRH3_list)):
                    if (len(CDRH3_list)%2)==0:
                        CDRH3_list.insert(int(len(CDRH3_list)/2),'-')
                    else:
                        CDRH3_list.insert(int((len(CDRH3_list)-1)/2),'-')

                result_CDRH3.append(CDRH3_list)
            else:
                CDRH3_extra.append(i)
        CDRH3_align=pd.DataFrame(result_CDRH3)    
        CDRH3_rename=['H93','H94','H95','H96','H97','H98','H99','H99A','H99B','H99C','H99D','H99E','H100S','H100T','H100U','H100V','H100W','H100X','H100Y','H100Z','H100','H101','H102']
        CDRH3_align.columns=CDRH3_rename
        CDRH3_align.index=VH_F1.iloc[CDRH3_noextra,:].index

        VH_chain=pd.concat([VH_F1,CDRH1_align,VH_F2,CDRH3_align,VH_F3],axis=1,sort=False).drop(mab_num_cdrh1, axis=0).dropna()
        VH_align=VH_chain.loc[:, ~(VH_chain.astype(str) == '-').all()]
        
    VH_all.append(VH_align)




In [ ]:
Fv_all=[]
for i in range(0,len(val)):
    x=pd.concat([position,pd.concat([VH_all[i],VL_all[i]],axis=1,sort=False).dropna()],axis=0,sort=False).fillna('-')
    x.iloc[:,0]=x.index
    Fv_all.append(x)
    
frame=[]
for i in range(0,len(val)):
    frame.append(Fv_all[i])
    
neg_master=pd.concat(frame,axis=0,sort=False)
neg_master=neg_master.reset_index().iloc[:,2:]

In [ ]:
max_length=[130,113]
min_length=[112,104]
neg_master_VL=neg_master.iloc[:,neg_master.T.index.tolist().index('L1'):neg_master.T.index.tolist().index('L107')]
neg_master_VH=neg_master.iloc[:,neg_master.T.index.tolist().index('H1'):neg_master.T.index.tolist().index('H113')]
neg_master_VL_len=neg_master_VL.replace('-','').apply(lambda x: ''.join(x), axis = 1) 
neg_master_VH_len=neg_master_VH.replace('-','').apply(lambda x: ''.join(x), axis = 1) 
len_inchespace=[]
for i in range(0,len(neg_master)):
    print(i)
    if len(neg_master_VH_len[i])<=130 and len(neg_master_VH_len[i])>=112 and len(neg_master_VL_len[i])<=113 and len(neg_master_VL_len[i])>=104:
        len_inchespace.append(i)
        
neg_result=pd.concat([position,neg_master.iloc[len_inchespace,:]],axis=0,sort=False)
neg_result.iloc[:,0]=neg_result.index

In [ ]:
## Calculate germline and CDR length
neg_result=neg_result.reset_index().iloc[:,1:]
neg_VL_seq=neg_result.iloc[:,neg_result.T.index.tolist().index('L1'):neg_result.T.index.tolist().index('L107')+1]
neg_VH_seq=neg_result.iloc[:,neg_result.T.index.tolist().index('H1'):neg_result.T.index.tolist().index('H113')+1]
neg_VL_seq=neg_VL_seq.replace('-','').apply(lambda x: ''.join(x), axis = 1) 
neg_VH_seq=neg_VH_seq.replace('-','').apply(lambda x: ''.join(x), axis = 1) 

In [ ]:
import os
import re
from anarci import number
from anarci import anarci
seq_VH=[]
seq_VL=[]
name_list=list(neg_VL_seq.index)
for i in range(0,len(neg_VL_seq)):
    seq_VH+=[[(name_list[i],neg_VH_seq[i])]]
    seq_VL+=[[(name_list[i],neg_VL_seq[i])]]
germline_VH=[]
germline_VL=[]
none_type=[]
for i in range(0,len(neg_VL_seq)):
    print(i)
    A,B_VH,C=anarci(seq_VH[i], scheme=("kabat"),assign_germline=True)    
    A,B_VL,C=anarci(seq_VL[i], scheme=("kabat"),assign_germline=True)
    if B_VH==[None] or B_VL==[None]:
        none_type.append(i)
    else:
        c.append([B_VH[0][0]['germlines']['v_gene'][0][1],B_VH[0][0]['germlines']['j_gene'][0][1]])
        germline_VL.append([B_VL[0][0]['germlines']['v_gene'][0][1],B_VL[0][0]['germlines']['j_gene'][0][1]])

In [ ]:
LCDR1=neg_result.iloc[:,neg_result.T.index.tolist().index('L24'):neg_result.T.index.tolist().index('L34')+1]
LCDR2=neg_result.iloc[:,neg_result.T.index.tolist().index('L50'):neg_result.T.index.tolist().index('L56')+1]
LCDR3=neg_result.iloc[:,neg_result.T.index.tolist().index('L89'):neg_result.T.index.tolist().index('L97')+1]
HCDR1=neg_result.iloc[:,neg_result.T.index.tolist().index('H26'):neg_result.T.index.tolist().index('H35')+1]
HCDR2=neg_result.iloc[:,neg_result.T.index.tolist().index('H50'):neg_result.T.index.tolist().index('H65')+1]
HCDR3=neg_result.iloc[:,neg_result.T.index.tolist().index('H95'):neg_result.T.index.tolist().index('H102')+1]


LCDR=[LCDR1,LCDR2,LCDR3]
HCDR=[HCDR1,HCDR2,HCDR3]
LCDR_length=[]
HCDR_length=[]
for i in range(0,3):
    LCDR_length.append(LCDR[i].replace('-','').apply(lambda x: ''.join(x), axis = 1).str.len())
    HCDR_length.append(HCDR[i].replace('-','').apply(lambda x: ''.join(x), axis = 1).str.len())
    
 